<a href="https://colab.research.google.com/github/abrichr/visual-contact-tracing/blob/master/CTAI_MOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd ~
# https://github.com/Zhongdao/Towards-Realtime-MOT

! git clone https://github.com/Zhongdao/Towards-Realtime-MOT
%cd Towards-Realtime-MOT

In [ ]:
! pip install motmetrics lap cython_bbox youtube-dl imageio

In [ ]:
! youtube-dl https://www.youtube.com/watch?v=ptzKx9voJBY -o video.mp4

In [ ]:
def download_file_from_google_drive(file_id, file_name):
  # download a file from the Google Drive link
  !rm -f ./cookie
  !curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=$file_id" > /dev/null
  confirm_text = !awk '/download/ {print $NF}' ./cookie
  confirm_text = confirm_text[0]
  !curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=$confirm_text&id=$file_id" -o $file_name
  with open(file_name, 'rb') as f:
    data = f.read()
    print('downloaded', len(data), 'bytes to', file_name)

# From https://github.com/Zhongdao/Towards-Realtime-MOT/issues/99
file_id_name_tups = [
  #('1nlnuYfGNuHWZztQHXwVZSL_FvfE551pA', 'jde.1088x608.uncertainty.pt'),
  ('1UKgkYrsV-59kYaHgWeJ70p5Mij3QWuFr', 'jde_864x480_uncertainty.pt'),
  #('1sca65sHMnxY7YJ89FJ6Dg3S3yAjbLdMz', 'jde_576x320_uncertainty.pt'),
]
for file_id, file_name in file_id_name_tups:
  download_file_from_google_drive(file_id, file_name)

In [ ]:
import imageio

ffmpeg = imageio.plugins.ffmpeg                                                
try:                                                                           
    ffmpeg.download()                                                          
except:                                                                        
    pass                                                                       
ffmpeg_exe = ffmpeg.get_exe()

video_filename = 'video.mp4'
video_filename_reenc = video_filename + '-reenc.avi'
start_time_seconds = 0
duration_seconds = 10

cmd_parts = [
  ffmpeg_exe,
  '-i', video_filename,
  '-vcodec', 'h264',
  '-acodec', 'aac',
  #'-c', 'copy',
  '-strict',
  '-2',
  '-ss', str(start_time_seconds),
  '-t', str(duration_seconds),
  '-y',
  '-loglevel', 'debug',
  '-an',
  # 640 = 1280 / 2
  #'-vf',  'scale=640:-1',
  '"' + video_filename_reenc + '"'
]
cmd = ' '.join(cmd_parts)
print('Running cmd:\n', cmd)                                
! $cmd                                

with open(video_filename_reenc, 'rb') as f:
  data = f.read()
  print('wrote', len(data), 'bytes')

In [ ]:
!python demo.py --input-video $video_filename_reenc --cfg "./cfg/yolov3_576x320.cfg" --weights "./jde_576x320_uncertainty.pt"  --output-format video --output-root "output" \
   --min-box-area 100 --conf-thres 0.1 --iou-thres 0.1

Namespace(cfg='./cfg/yolov3_576x320.cfg', conf_thres=0.1, input_video='video.mp4-reenc.avi', iou_thres=0.1, min_box_area=100.0, nms_thres=0.4, output_format='video', output_root='output', track_buffer=30, weights='./jde_576x320_uncertainty.pt')

2020-07-20 22:01:22 [INFO]: Starting tracking...
Lenth of the video: 300 frames
2020-07-20 22:01:26 [INFO]: Processing frame 0 (100000.00 fps)
/root/Towards-Realtime-MOT/tracker/multitracker.py:142: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "tlbr_to_tlwh" failed type inference due to: non-precise type pyobject
[1] During: typing of argument at /root/Towards-Realtime-MOT/tracker/multitracker.py (145)

File "tracker/multitracker.py", line 145:
    def tlbr_to_tlwh(tlbr):
        ret = np.asarray(tlbr).copy()
        ^

  @staticmethod
/usr/local/lib/python3.6/dist-packages/numba/object_mode_passes.py:178: NumbaWarning: Function "tlbr_to_tlwh" was compiled in object mode without forceobj=Tr

In [ ]:
!ls -alh output

total 30M
drwxr-xr-x  3 root root 4.0K Jul 20 21:52 .
drwxr-xr-x 11 root root 4.0K Jul 20 21:51 ..
drwxr-xr-x  2 root root  12K Jul 20 21:52 frame
-rw-r--r--  1 root root  29M Jul 20 22:03 result.mp4
-rw-r--r--  1 root root 1.6M Jul 20 22:01 results.txt


In [ ]:
from google.colab import files
# doesn't open in QuickTime, but works in VLC
files.download('./output/result.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>